In [ ]:
# Chuỗi nào có PM thì giá trị giờ cộng thêm 12 phút
def process_time(gold_time: str):
  if gold_time.endswith("PM"):
    parts = gold_time.split(":")
    hour = int(parts[0]) + 12
    gold_time = f"{hour}:{parts[1]}"

  res = re.split("(AM|PM)", gold_time)[0].strip()
  res = res.split(":")[0]
  return res

format = r"\d+:\d+\s*(AM|PM)"

df_ordinal_cols = df[ordinal_cols]
index_not_satisfy_format = df_ordinal_cols[df_ordinal_cols.applymap(lambda item: re.fullmatch(format, item) is None)].stack().index

# Get các giá trị ứng với hiện tượng có xảy ra
df_ordinal_cols_happen_stack = df_ordinal_cols.stack()
df_ordinal_cols_happen_stack = df_ordinal_cols_happen_stack[~df_ordinal_cols_happen_stack.index.isin(index_not_satisfy_format)]
df_ordinal_cols_happen_stack = df_ordinal_cols_happen_stack.apply(lambda item: process_time(item))

# Cập nhật
df_ordinal_cols_stack = df_ordinal_cols.stack()
df_ordinal_cols_stack[df_ordinal_cols_happen_stack.index] = df_ordinal_cols_happen_stack
df[ordinal_cols] = df_ordinal_cols_stack.unstack()


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Các mô hình cơ bản (base learners)
base_learners = [
    ('decision_tree', DecisionTreeClassifier(max_depth=3)),
    ('svm', SVC(kernel='linear', probability=True))
]

# Mô hình học kết hợp (meta-learner)
meta_learner = LogisticRegression()

# Khởi tạo StackingClassifier
stacking_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=5)

# Tải dữ liệu mẫu
data = load_iris()
X = data.data
y = data.target

# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình
stacking_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = stacking_model.predict(X_test)

# Tính độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 1.0000


In [ ]:
from sklearn.preprocessing import 